# Proyecto Sprint 13: Métodos Numéricos

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

Iniciaremos con la carga de librerías para el proyecto.

### Inicialización

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from lightgbm import LGBMRegressor
from catboost import Pool, CatBoostRegressor



Continuamos con la carga del dataset.


### Carga de Datos

In [2]:
df = pd.read_csv("/datasets/car_data.csv")
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,21/03/2016 09:50,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,21/03/2016 00:00,0,2694,21/03/2016 10:42
354365,14/03/2016 17:48,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,14/03/2016 00:00,0,39576,06/04/2016 00:46
354366,05/03/2016 19:56,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,05/03/2016 00:00,0,26135,11/03/2016 18:17
354367,19/03/2016 18:57,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,19/03/2016 00:00,0,87439,07/04/2016 07:15


De forma inicial vemos que se tienen datos incompletos y que se puede mejorar los nombres de las columnas para mejor manejo posteriormente.

### Preparación de los Datos

Comenzamos modificando los nombres de las columnas para dejarlas en minúsculas.

In [3]:
df.columns = df.columns.str.lower()
df

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,mileage,registrationmonth,fueltype,brand,notrepaired,datecreated,numberofpictures,postalcode,lastseen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,21/03/2016 09:50,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,21/03/2016 00:00,0,2694,21/03/2016 10:42
354365,14/03/2016 17:48,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,14/03/2016 00:00,0,39576,06/04/2016 00:46
354366,05/03/2016 19:56,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,05/03/2016 00:00,0,26135,11/03/2016 18:17
354367,19/03/2016 18:57,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,19/03/2016 00:00,0,87439,07/04/2016 07:15


In [4]:
df[df.duplicated()]

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,mileage,registrationmonth,fueltype,brand,notrepaired,datecreated,numberofpictures,postalcode,lastseen
14266,21/03/2016 19:06,5999,small,2009,manual,80,polo,125000,5,petrol,volkswagen,no,21/03/2016 00:00,0,65529,05/04/2016 20:47
27568,23/03/2016 10:38,12200,bus,2011,manual,125,zafira,40000,10,gasoline,opel,no,23/03/2016 00:00,0,26629,05/04/2016 07:44
31599,03/04/2016 20:41,4950,wagon,2003,auto,170,e_klasse,150000,4,gasoline,mercedes_benz,no,03/04/2016 00:00,0,48432,05/04/2016 21:17
33138,07/03/2016 20:45,10900,convertible,2005,auto,163,clk,125000,5,petrol,mercedes_benz,no,07/03/2016 00:00,0,61200,21/03/2016 03:45
43656,13/03/2016 20:48,4200,sedan,2003,manual,105,golf,150000,10,gasoline,volkswagen,no,13/03/2016 00:00,0,14482,13/03/2016 20:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349709,03/04/2016 20:52,700,small,1999,manual,60,ibiza,150000,12,petrol,seat,yes,03/04/2016 00:00,0,6268,05/04/2016 21:47
351555,26/03/2016 16:54,3150,bus,2003,manual,86,transit,150000,11,gasoline,ford,no,26/03/2016 00:00,0,96148,02/04/2016 07:47
352384,15/03/2016 21:54,5900,wagon,2006,manual,129,3er,150000,12,petrol,bmw,no,15/03/2016 00:00,0,92526,20/03/2016 21:17
353057,05/03/2016 14:16,9500,small,2013,manual,105,ibiza,40000,5,petrol,seat,no,04/03/2016 00:00,0,61381,05/04/2016 19:18


Vemos que tenemos filas completamente duplicadas, en este caso es muy relevante saber si es efectivamente por una mala muestra de la toma de datos o porque se produce alguna similitud en los elementos tomados. En este caso, revisando, tenemos que, por ejemplo, su postalcode se repite (y el cual debiese ser único para cada usuario). Por ende lo más seguro es que la captura de datos tuvo algun error en su consolidación, por lo que se procederá a eliminar las filas repetidas.

In [5]:
df[df["datecrawled"] == "21/03/2016 19:06"]

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,mileage,registrationmonth,fueltype,brand,notrepaired,datecreated,numberofpictures,postalcode,lastseen
183,21/03/2016 19:06,5999,small,2009,manual,80,polo,125000,5,petrol,volkswagen,no,21/03/2016 00:00,0,65529,05/04/2016 20:47
947,21/03/2016 19:06,1450,bus,2000,manual,82,vito,150000,6,gasoline,mercedes_benz,no,21/03/2016 00:00,0,55120,23/03/2016 11:32
12462,21/03/2016 19:06,6300,small,2007,manual,95,one,90000,7,petrol,mini,no,21/03/2016 00:00,0,60318,29/03/2016 11:47
14266,21/03/2016 19:06,5999,small,2009,manual,80,polo,125000,5,petrol,volkswagen,no,21/03/2016 00:00,0,65529,05/04/2016 20:47
134813,21/03/2016 19:06,6500,small,2005,manual,116,cooper,150000,5,petrol,mini,no,21/03/2016 00:00,0,61279,05/04/2016 20:47
166105,21/03/2016 19:06,1599,small,2004,manual,75,clio,150000,2,petrol,renault,no,21/03/2016 00:00,0,57635,05/04/2016 21:18
172362,21/03/2016 19:06,8000,wagon,2004,auto,177,5er,150000,6,gasoline,bmw,NaN,21/03/2016 00:00,0,65719,02/04/2016 21:51
180614,21/03/2016 19:06,1000,small,2004,manual,0,other,150000,0,petrol,mitsubishi,NaN,21/03/2016 00:00,0,35584,05/04/2016 20:47
204526,21/03/2016 19:06,6499,wagon,2007,manual,170,passat,150000,0,gasoline,volkswagen,NaN,21/03/2016 00:00,0,65549,22/03/2016 15:22
265807,21/03/2016 19:06,1700,sedan,1996,manual,192,5er,150000,5,NaN,bmw,no,21/03/2016 00:00,0,63065,22/03/2016 10:42


In [6]:
df.drop_duplicates(inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354107 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   datecrawled        354107 non-null  object
 1   price              354107 non-null  int64 
 2   vehicletype        316623 non-null  object
 3   registrationyear   354107 non-null  int64 
 4   gearbox            334277 non-null  object
 5   power              354107 non-null  int64 
 6   model              334406 non-null  object
 7   mileage            354107 non-null  int64 
 8   registrationmonth  354107 non-null  int64 
 9   fueltype           321218 non-null  object
 10  brand              354107 non-null  object
 11  notrepaired        282962 non-null  object
 12  datecreated        354107 non-null  object
 13  numberofpictures   354107 non-null  int64 
 14  postalcode         354107 non-null  int64 
 15  lastseen           354107 non-null  object
dtypes: int64(7), object(

Se remueven las 262 filas repetidas, que simplemente incluirian un sesgo en nuestro análisis y posterior modelo.

Continuando, hay ciertas columnas que no presentan ningún valor para los modelos, por ende modificaremos nuestro dataset.

- DateCrawled: No nos aporta algo esencial.
- DateCreated: No aporta datos relevantes.
- PostalCode: No posee algo relevante respecto al objetivo buscado
- LastSeen: Tampoco posee relevancia para el análisis.
- Numberofpictures: todos sus valores son 0, no posee ningún aporte esencial.

In [7]:
non_relevant = ["datecrawled", "datecreated", "postalcode", "lastseen", "numberofpictures"]
df.drop(non_relevant, axis = 1, inplace = True)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354107 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   price              354107 non-null  int64 
 1   vehicletype        316623 non-null  object
 2   registrationyear   354107 non-null  int64 
 3   gearbox            334277 non-null  object
 4   power              354107 non-null  int64 
 5   model              334406 non-null  object
 6   mileage            354107 non-null  int64 
 7   registrationmonth  354107 non-null  int64 
 8   fueltype           321218 non-null  object
 9   brand              354107 non-null  object
 10  notrepaired        282962 non-null  object
dtypes: int64(5), object(6)
memory usage: 32.4+ MB


,price,vehicletype,registrationyear,gearbox,power,model,mileage,registrationmonth,fueltype,brand,notrepaired
0,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
...,...,...,...,...,...,...,...,...,...,...,...
354364,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes
354365,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no
354367,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no


Revisaremos ahora sus valores ausentes.

In [8]:
df["vehicletype"].value_counts(dropna = False)

sedan          91399
small          79753
wagon          65115
NaN            37484
bus            28752
convertible    20180
coupe          16147
suv            11991
other           3286
Name: vehicletype, dtype: int64

In [9]:
df["gearbox"].value_counts(dropna = False)

manual    268034
auto       66243
NaN        19830
Name: gearbox, dtype: int64

In [10]:
df["model"].value_counts(dropna = False)

golf                  29215
other                 24402
3er                   19744
NaN                   19701
polo                  13057
                      ...  
i3                        8
serie_3                   4
rangerover                4
range_rover_evoque        2
serie_1                   2
Name: model, Length: 251, dtype: int64

In [11]:
df["fueltype"].value_counts(dropna = False)

petrol      216161
gasoline     98658
NaN          32889
lpg           5307
cng            565
hybrid         233
other          204
electric        90
Name: fueltype, dtype: int64

In [12]:
df["notrepaired"].value_counts(dropna = False)

no     246927
NaN     71145
yes     36035
Name: notrepaired, dtype: int64

Todas las columnas previas poseen valores ausentes. Ahora, son variables categóricas y, por ende, debemos decidir qué hacer con ellas. Para todas las columnas, se decide rellenar con "non_specified".

In [13]:
string_column_names = df.select_dtypes(object).columns
df[string_column_names] = df[string_column_names].fillna('non_specified')

Comprobando para el dataset en general:

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354107 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   price              354107 non-null  int64 
 1   vehicletype        354107 non-null  object
 2   registrationyear   354107 non-null  int64 
 3   gearbox            354107 non-null  object
 4   power              354107 non-null  int64 
 5   model              354107 non-null  object
 6   mileage            354107 non-null  int64 
 7   registrationmonth  354107 non-null  int64 
 8   fueltype           354107 non-null  object
 9   brand              354107 non-null  object
 10  notrepaired        354107 non-null  object
dtypes: int64(5), object(6)
memory usage: 32.4+ MB


Ahora, tenemos muchas columnas categóricas en adición a las numéricas, por ende debemos codificar las mismas para el posterior análisis y entrenamiento de los modelos.

### Codificación de los datos


Para codificar nuestros datos categóricos, debemos decidir que técnica utilizar. Cuando las categorías son bastante extensas, lo mejor es utilizar el Ordinal Encoder, debido a que si, por ejemplo, utilizaramos One-hot encoding tendríamos muchas columnas adicionales lo que haría el proceso de modelamiento de nuestros algoritmos de ML mucho más lentos, y lo que se busca es eficiencia.

In [15]:
encoder = OrdinalEncoder()
data_ordinal = pd.DataFrame(encoder.fit_transform(df), columns=df.columns)
data_ordinal

,price,vehicletype,registrationyear,gearbox,power,model,mileage,registrationmonth,fueltype,brand,notrepaired
0,256.0,3.0,86.0,1.0,0.0,116.0,12.0,0.0,7.0,38.0,1.0
1,3587.0,2.0,104.0,1.0,190.0,161.0,11.0,5.0,2.0,1.0,2.0
2,2590.0,7.0,97.0,0.0,163.0,117.0,11.0,8.0,2.0,14.0,1.0
3,696.0,6.0,94.0,1.0,75.0,116.0,12.0,6.0,7.0,38.0,0.0
4,1333.0,6.0,101.0,1.0,69.0,101.0,9.0,7.0,2.0,31.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
354102,0.0,3.0,98.0,1.0,0.0,78.0,12.0,7.0,7.0,22.0,2.0
354103,923.0,3.0,98.0,2.0,0.0,161.0,2.0,1.0,5.0,33.0,1.0
354104,568.0,1.0,93.0,0.0,101.0,106.0,11.0,3.0,7.0,32.0,0.0
354105,2492.0,0.0,89.0,1.0,102.0,225.0,12.0,3.0,2.0,38.0,0.0


### Segmentación de los Datos

Procederemos a separar los datos de entrenamiento y test.

In [16]:
df_train, df_valid = train_test_split(data_ordinal, test_size = 0.2, random_state = 1702)
print(df_train.shape)
print(df_valid.shape)

(283285, 11)
(70822, 11)


In [17]:
features_train = df_train.drop('price', axis=1)
target_train = df_train['price']
features_valid = df_valid.drop('price', axis=1)
target_valid = df_valid["price"]

### Estandarización de los Datos

Utilizaremos la estandarización de nuestros datos para ajustarlos a una medida que no afecte el algoritmo en base a sus dimensiones individuales por columna. Es decir, que todos sean proporcionalmente iguales entre sí. Ahora, se elige la estandarización porque tiene muy buenos resultados con algoritmos basados en gradiente, que es algo que utilizaremos como modelos en los siguientes apartados.

In [18]:
scaler = StandardScaler()
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)


 Habiendo culminado este último paso, podemos pasar a entrenar los modelos mediante sus algoritmos respectivos.

## Entrenamiento del modelo 

Habiendo preprocesado todos los datos, comenzaremos con los respectivos algoritmos de entrenamiento de los diferentes modelos.

### Regresión Lineal

La regresion lineal es bastante limitada en cuanto a sus hiperparámetros, por lo que efectuamos su algoritmo y entrenamos los datos.

In [19]:
%%time
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)

result = mean_squared_error(target_valid,predictions)**0.5
print("RMSE del modelo es:", result.round(2))

print("media predicciones",predictions.mean().round(2))
print("media real", target_valid.mean().round(2))


RMSE del modelo es: 669.1
media predicciones 1316.17
media real 1317.01
CPU times: user 50.3 ms, sys: 63.8 ms, total: 114 ms
Wall time: 97.9 ms


### Bosque Aleatorio

Partimos haciendo una prueba para revisar cuáles serían los mejores parámetros para nuestros datos, utilizando para ello GridSeach Cross Validation (por el tiempo que se demoró en efectuarlo, dejaré aquello en la celda como comentario).

In [20]:
#%%time

#param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 5, 10]}

## Se crea un modelo para random forest
# model2 = RandomForestRegressor(random_state=1702)
# se genera un grid search para sus hiperparametros
#grid_search = GridSearchCV(model2, param_grid, cv=5, scoring='neg_mean_squared_error')
#grid_search.fit(features_train, target_train)

# Para obtener nuestros resultados
#best_params = grid_search.best_params_
#best_score = -grid_search.best_score_

#print("Best Parameters:", best_params)
#print("Best Score (MSE):", best_score)

Los parametros que arrojaron fueron los siguientes:

- Best Parameters: {'max_depth': None, 'n_estimators': 200}
- Best Score (MSE): 150904.91533997003

- CPU times: user 40min 53s, sys: 13.55 s, total: 41min 1s
- Wall time: 41min 1s

Considerar que al intentar con n_estimators superiores a 200 el kernel simplemente dejaba de funcionar.

In [21]:
%%time
est = 200
model2 = RandomForestRegressor(random_state=1702, n_estimators=est, max_depth = None) 
model2.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
predicted = model2.predict(features_valid)
rmse = mean_squared_error(target_valid,predicted)**0.5
print("El RMSE del modelo es (n_estimators = ", est,"):", rmse)
print("media predicciones",predicted.mean().round(2))
print("media real", target_valid.mean().round(2))
print()


El RMSE del modelo es (n_estimators =  200 ): 380.88009060130827
media predicciones 1316.83
media real 1317.01

CPU times: user 2min 22s, sys: 1.21 s, total: 2min 24s
Wall time: 2min 24s


Se logra reducir el RMSE con respecto a la regresión lineal prácticamente a la mitad. Con esto, sumado a la modificación de hiperparámetros, se puede decir que vamos por buen camino.

### Árbol de Decisión

Efectuando la misma cross validación para el árbol, tenemos:

Se deja en comentario por el tiempo que lleva efectuarla.

In [25]:
#param_grid = {
#    'max_depth': [10, 20, 30, None],
#    'min_samples_split': [2, 5, 10],
#    'min_samples_leaf': [1, 2, 4]
#}
#dtree_reg = DecisionTreeRegressor(random_state=1702) # Initialize a decision tree regressor
#grid_search = GridSearchCV(estimator=dtree_reg, param_grid=param_grid, 
#                           cv=5, n_jobs=-1, verbose=0, scoring='neg_mean_squared_error')
#grid_search.fit(features_train, target_train)
#best_dtree_reg = grid_search.best_estimator_
#predicted_tree = best_dtree_reg.predict(features_valid)
#mse = mean_squared_error(target_valid, predicted_tree)
#rmse = mse ** 0.5
#best_params = grid_search.best_params_
#print(f"Best parameters: {best_params}")
#print(f"Test RMSE: {rmse}")

Utilizaremos parámetros cercanos para nuestro modelo:

- max_depth = 16
- min_samples_leaf = 4
- min_samples_split = 10

Ya que el proceso se demoraba un poco más, manualmente encontramos que a una profundidad de 16 existe un mejor valor del RMSE por lo que nos quedaremos con aquel. Lo cual, independientemente del caso, no supera al RMSE del árbol aleatorio.

In [26]:
%%time
model3 = DecisionTreeRegressor(random_state = 1702, max_depth =16, min_samples_leaf = 4, min_samples_split = 10)  
model3.fit(features_train, target_train) 
predictions_tree = model3.predict(features_valid)
rmse2 = mean_squared_error(target_valid,predictions_tree)**0.5

print("El RMSE del modelo es", rmse2)
print("media predicciones",predictions_tree.mean().round(2))
print("media real", target_valid.mean().round(2))
print()

El RMSE del modelo es 425.21539361427494
media predicciones 1316.17
media real 1317.01

CPU times: user 818 ms, sys: 22 µs, total: 818 ms
Wall time: 817 ms


Su RMSE no es mejor que le árbol, pero siempre mejor que la regresión lineal.

### LightGBM

Dejaré el codigo utilizado para verificar sus mejores parámetros como comentario con el resultado al final ya que tomó bastante tiempo.

In [ ]:
#estimator = lgb.LGBMRegressor(num_leaves=31)

#param_grid = {"learning_rate": [0.1, 0.5, 1], "n_estimators": [1000, 1100, 1200,1300,1400,1500]}

#gbm = GridSearchCV(estimator, param_grid, cv=3)
#gbm.fit(features_train, target_train)

#print(f"Los mejores parámetros de GridSearch son: {gbm.best_params_}")

###Resultado###
#Los mejores parámetros de GridSearch son: {'learning_rate': 0.1, 'n_estimators': 1500}


In [27]:
%%time
model4 = LGBMRegressor(random_state= 1702,num_leaves=31, learning_rate=0.1, n_estimators=1500)

# Train the model using the training data.
model4.fit(features_train, target_train)

# Make predictions on the training and validation data.
predictions_lgb = model4.predict(features_valid, num_iteration= model4.best_iteration_)
rmse3 = mean_squared_error(target_valid,predictions_lgb)**0.5


print("El RMSE del modelo es", rmse3)
print("media predicciones",predictions_lgb.mean().round(2))
print("media real", target_valid.mean().round(2))
print()

El RMSE del modelo es 367.1770308304506
media predicciones 1316.51
media real 1317.01

CPU times: user 30 s, sys: 135 ms, total: 30.1 s
Wall time: 30.1 s


Hasta ahora LightGBM mediante nuestro proceso ha sido el que ha obtenido mejores resultados. Veamos ahora CatBoost

### CatBoost

In [28]:
eval_dataset = Pool(features_valid, target_valid)
model5 = CatBoostRegressor(learning_rate = 0.03 , eval_metric='RMSE')
model5.fit(features_train,target_train, eval_set=eval_dataset, verbose=False)

print(model5.get_best_iteration())

999


Nos señala que con esos parámetros su mejor iteración es 999. 

In [40]:
%%time
train_pool = Pool(features_train, target_train)
target_pool = Pool(features_valid)

model5 = CatBoostRegressor(random_state = 1702, iterations=999, depth=6, 
                           learning_rate=0.6, loss_function='RMSE', silent= True)
model5.fit(train_pool)
preds = model5.predict(target_pool)
rmse4 = mean_squared_error(target_valid,preds)**0.5

print("El RMSE del modelo es", rmse4)
print("media predicciones",preds.mean().round(2))
print("media real", target_valid.mean().round(2))
print()

El RMSE del modelo es 373.99884017688805
media predicciones 1316.1
media real 1317.01

CPU times: user 30.6 s, sys: 27.9 ms, total: 30.6 s
Wall time: 30.7 s


Con esto, detendremos el modelo, ya que se obtuvo un tiempo similar al algoritmo LightGBM.

## Análisis del modelo

Para determinar nuestro mejor modelo haremos un resumen:

                           RMSE         Tiempo de Ejecución
                           
    - Regresión Lineal :   669.1        114ms (milisegundos)
    - Bosque Aleatorio :   380.88       2min 24s 
    - Árbol de Decisión:   425.21       818ms
    - LightGBM         :   367.18       30.1s
    - Catboost         :   373.99.      30.6s

Considerando lo anterior, mediante el manejo de hiperparámetros y el valor de la raíz del error cuadrático medio, nuestro algoritmo para modelar el dataset y entrenarlo sería **LightGBM**.

Su mismo nombre lo señala, light de liviano porque según sus propias pruebas mostradas en la documentación es un algoritmo de descenso de gradiente muy rápido y tiene mejor rendimiento que otros en la misma categoría.

Ahora, debo insistir, que ya que los hiperparámetros seleccionados podrían haber sido optimizados aun más, puede que los resultados variasen de efectuar más pruebas. 
